In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import pymysql

In [7]:
conn = pymysql.connect(host="192.168.56.101",user='root', passwd="123456", db='scrapt', charset='utf8')#创建一个连接

In [12]:
cur = conn.cursor()#创建一个游标，相当于一个对象，如果要对多个表进行操作可以创建多个对象
cur.execute("USE scraping")#对象.方法（）  使用scraping表格
random.seed(datetime.datetime.now())
def store(title, content):#类似的使用对象.方法（）来操作数据库
    cur.execute("INSERT INTO pages (title, content) VALUES (\"%s\",\"%s\")", (title, content))
    cur.connection.commit()
def getLinks(articleUrl): #爬取数据
    html = urlopen("http://en.wikipedia.org"+articleUrl)
    bsObj = BeautifulSoup(html)
    title = bsObj.find("h1").find("span").get_text()
    content = bsObj.find("div", {"id":"mw-content-text"}).find("p").get_text()
    store(title, content)
    return bsObj.find("div", {"id":"bodyContent"}).findAll("a",href=re.compile("^(/wiki/)((?!:).)*$"))
links = getLinks("/wiki/Kevin_Bacon")
try:
    while len(links) > 0:
        newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
        print(newArticle)
        links = getLinks(newArticle)
finally:
    cur.close()#使用完数据库操作后要及时关闭游标和连接，因为游标和连接不会自动关闭，这样能避免数据库出现问题
    conn.close()

URLError: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>